In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


In [2]:
train = pd.read_csv('../../../Kaggle_input/input/ctd_datawiz-2023/train.csv')
test = pd.read_csv('../../../Kaggle_input/input/ctd_datawiz-2023/test.csv')
sample_submission = pd.read_csv('../../../Kaggle_input/input/ctd_datawiz-2023/sample_submission.csv')

In [6]:
train.sample(5)

,User ID,Mobile No.,User DOB,User Job,Duration,Intl_Plan,Voicemail_Plan,Voicemail_Messages,Day_Minutes,Day_Calls,...,Eve_Calls,Eve_Charge,Night_Minutes,Night_Calls,Night_Charge,Intl_Minutes,Intl_Calls,Intl_Charge,Service_Calls,Attrition
2284,2833,7991482064,1963-01-05,"Teacher, primary school",87,no,no,0.0,124.3,91.0,...,105,191.62,256.3,109,149.89,7.5,5,26.39,3,No
3610,4386,9998407540,1977-06-17,Clinical research associate,134,no,yes,32.0,224.0,75.0,...,103,107.77,211.6,80,123.76,10.7,5,NaN,2,No
2458,3042,8746541967,1960-05-11,Site engineer,39,no,no,0.0,93.3,83.0,...,114,220.61,206.2,104,120.64,6.5,4,22.88,0,No
2537,3132,6129136221,1947-07-12,"Surveyor, commercial/residential",107,NaN,no,0.0,189.7,76.0,...,65,172.51,244.0,91,142.74,8.3,3,29.12,5,No
759,1036,9256287533,1957-08-20,"Research officer, trade union",119,no,no,0.0,190.4,74.0,...,113,238.29,161.2,111,94.25,10.0,1,35.10,2,No


In [8]:
test.sample(5)

,User ID,Mobile No.,User DOB,User Job,Duration,Intl_Plan,Voicemail_Plan,Voicemail_Messages,Day_Minutes,Day_Calls,...,Eve_Minutes,Eve_Calls,Eve_Charge,Night_Minutes,Night_Calls,Night_Charge,Intl_Minutes,Intl_Calls,Intl_Charge,Service_Calls
799,346,9451970163,1995-04-11,Chartered accountant,141,no,no,0,260.2,131,...,179.2,120,197.99,135.0,119,79.04,7.2,8,25.22,3
155,1270,6250806117,1973-10-05,Film/video editor,55,no,yes,45,130.5,114,...,208.4,94,230.23,141.6,114,82.81,11.0,5,38.61,4
384,2711,9364048214,1990-11-24,International aid/development worker,63,no,no,0,261.8,69,...,245.0,135,270.79,202.1,94,118.17,14.7,4,51.61,0
102,788,9155541504,1976-03-02,Financial planner,24,no,no,0,265.6,86,...,208.8,102,230.75,182.5,105,106.73,11.1,6,39.00,2
262,1935,9500054465,1992-07-05,Building control surveyor,105,no,no,0,281.3,124,...,301.5,96,333.19,202.8,109,118.69,8.7,3,30.55,0


In [9]:
sample_submission.head()

,User ID,Attrition
0,11,No
1,16,No
2,22,No
3,34,No
4,42,No


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4150 entries, 0 to 4149
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   User ID             4150 non-null   int64  
 1   Mobile No.          4150 non-null   int64  
 2   User DOB            4150 non-null   object 
 3   User Job            4150 non-null   object 
 4   Duration            4150 non-null   int64  
 5   Intl_Plan           3679 non-null   object 
 6   Voicemail_Plan      4150 non-null   object 
 7   Voicemail_Messages  3681 non-null   float64
 8   Day_Minutes         4150 non-null   float64
 9   Day_Calls           3640 non-null   float64
 10  Day_Charge          4150 non-null   float64
 11  Eve_Minutes         3626 non-null   float64
 12  Eve_Calls           4150 non-null   int64  
 13  Eve_Charge          4150 non-null   float64
 14  Night_Minutes       3548 non-null   float64
 15  Night_Calls         4150 non-null   int64  
 16  Night_

In [11]:
train.drop(['Mobile No.'], axis=1, inplace=True)
test.drop(['Mobile No.'], axis=1, inplace=True)

In [18]:
train['User DOB'] = pd.to_datetime(train['User DOB'])
test['User DOB'] = pd.to_datetime(test['User DOB'])

# let's create a new column 'age' from 'User DOB'
train['age'] = train['User DOB'].apply(lambda x: 2023 - x.year)
test['age'] = test['User DOB'].apply(lambda x: 2023 - x.year)

In [19]:
train.drop(['User DOB'], axis=1, inplace=True)
test.drop(['User DOB'], axis=1, inplace=True)

In [25]:
train['Attrition'].value_counts()

No     3943
Yes     207
Name: Attrition, dtype: int64

In [21]:
def encode_cat(df, col, encoding_type):
    if encoding_type == 'label':
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col])
    elif encoding_type == 'onehot':
        df = pd.get_dummies(df, columns=[col], prefix=[col])
    return df

In [103]:
# Encoding variables in the train dataset and create a new dataframe called df_train_encoded
df_train_encoded = train.copy()
df_train_encoded = encode_cat(df_train_encoded, 'User Job', 'label')
df_train_encoded = encode_cat(df_train_encoded, 'Intl_Plan', 'onehot')
df_train_encoded = encode_cat(df_train_encoded, 'Voicemail_Plan', 'onehot')
df_train_encoded = encode_cat(df_train_encoded, 'Attrition', 'label')

In [104]:
df_train_encoded.head()

,User ID,User Job,Duration,Voicemail_Messages,Day_Minutes,Day_Calls,Day_Charge,Eve_Minutes,Eve_Calls,Eve_Charge,...,Intl_Minutes,Intl_Calls,Intl_Charge,Service_Calls,Attrition,age,Intl_Plan_no,Intl_Plan_yes,Voicemail_Plan_no,Voicemail_Plan_yes
0,3487,594,40,0.0,309.4,86.0,683.80,271.3,98,299.78,...,11.2,5,39.26,3,1,54,1,0,1,0
1,3488,491,161,0.0,159.6,102.0,352.69,212.6,119,234.91,...,12.6,7,NaN,5,1,42,1,0,1,0
2,3495,172,69,0.0,176.8,NaN,390.78,196.0,89,216.58,...,13.1,5,NaN,1,1,21,0,1,1,0
3,3499,462,114,0.0,115.6,93.0,255.45,NaN,78,180.31,...,9.7,1,34.06,2,1,47,0,1,1,0
4,3526,161,104,0.0,261.2,81.0,577.20,209.1,130,231.01,...,11.3,1,39.65,2,1,66,0,0,1,0


In [105]:
df_train_encoded['Attrition'].value_counts()

0    3943
1     207
Name: Attrition, dtype: int64

In [106]:
# Encoding variables in the test dataset and create a new dataframe called df_test_encoded
df_test_encoded = test.copy()
df_test_encoded = encode_cat(df_test_encoded, 'User Job', 'label')
df_test_encoded = encode_cat(df_test_encoded, 'Intl_Plan', 'onehot')
df_test_encoded = encode_cat(df_test_encoded, 'Voicemail_Plan', 'onehot')


In [107]:
df_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4150 entries, 0 to 4149
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   User ID             4150 non-null   int64  
 1   User Job            4150 non-null   int64  
 2   Duration            4150 non-null   int64  
 3   Voicemail_Messages  3681 non-null   float64
 4   Day_Minutes         4150 non-null   float64
 5   Day_Calls           3640 non-null   float64
 6   Day_Charge          4150 non-null   float64
 7   Eve_Minutes         3626 non-null   float64
 8   Eve_Calls           4150 non-null   int64  
 9   Eve_Charge          4150 non-null   float64
 10  Night_Minutes       3548 non-null   float64
 11  Night_Calls         4150 non-null   int64  
 12  Night_Charge        4150 non-null   float64
 13  Intl_Minutes        4150 non-null   float64
 14  Intl_Calls          4150 non-null   int64  
 15  Intl_Charge         3561 non-null   float64
 16  Servic

In [108]:
df_train_encoded.isna().sum()

User ID                 0
User Job                0
Duration                0
Voicemail_Messages    469
Day_Minutes             0
Day_Calls             510
Day_Charge              0
Eve_Minutes           524
Eve_Calls               0
Eve_Charge              0
Night_Minutes         602
Night_Calls             0
Night_Charge            0
Intl_Minutes            0
Intl_Calls              0
Intl_Charge           589
Service_Calls           0
Attrition               0
age                     0
Intl_Plan_no            0
Intl_Plan_yes           0
Voicemail_Plan_no       0
Voicemail_Plan_yes      0
dtype: int64

In [109]:
df_train_encoded.fillna(0, inplace=True)

In [110]:
df_test_encoded.isna().sum()

User ID               0
User Job              0
Duration              0
Voicemail_Messages    0
Day_Minutes           0
Day_Calls             0
Day_Charge            0
Eve_Minutes           0
Eve_Calls             0
Eve_Charge            0
Night_Minutes         0
Night_Calls           0
Night_Charge          0
Intl_Minutes          0
Intl_Calls            0
Intl_Charge           0
Service_Calls         0
age                   0
Intl_Plan_no          0
Intl_Plan_yes         0
Voicemail_Plan_no     0
Voicemail_Plan_yes    0
dtype: int64

In [179]:
df_train_encoded.corr()

,User ID,User Job,Duration,Voicemail_Messages,Day_Minutes,Day_Calls,Day_Charge,Eve_Minutes,Eve_Calls,Eve_Charge,...,Intl_Minutes,Intl_Calls,Intl_Charge,Service_Calls,Attrition,age,Intl_Plan_no,Intl_Plan_yes,Voicemail_Plan_no,Voicemail_Plan_yes
User ID,1.000000,-0.018209,-0.015583,-0.061174,0.040085,-0.055119,0.040085,-0.004727,0.007632,0.013831,...,0.024751,-0.025108,0.020761,0.057298,0.248596,-0.007719,-0.039264,0.028667,0.064085,-0.064085
User Job,-0.018209,1.000000,-0.007760,0.018949,0.004471,0.008527,0.004483,0.022186,-0.006600,0.022284,...,-0.010400,0.010913,0.015319,0.019498,0.007620,-0.000636,0.008637,-0.006997,-0.010758,0.010758
Duration,-0.015583,-0.007760,1.000000,-0.016811,-0.013972,0.032476,-0.013974,-0.004055,0.002685,-0.004202,...,-0.000318,0.008750,0.003109,-0.008856,0.008973,0.002683,0.009999,0.009813,0.007483,-0.007483
Voicemail_Messages,-0.061174,0.018949,-0.016811,1.000000,0.043315,0.013574,0.043312,0.014364,-0.000770,0.026448,...,0.001458,-0.002591,0.011080,0.005680,-0.073359,-0.022243,-0.005764,0.009067,-0.868450,0.868450
Day_Minutes,0.040085,0.004471,-0.013972,0.043315,1.000000,-0.009051,1.000000,-0.031549,0.002802,-0.062548,...,-0.031419,-0.026260,-0.012288,0.053861,0.144835,-0.025220,0.004091,0.017431,-0.043199,0.043199
Day_Calls,-0.055119,0.008527,0.032476,0.013574,-0.009051,1.000000,-0.009042,0.012167,-0.006315,-0.005810,...,0.022510,-0.030687,0.011257,-0.038237,-0.016250,-0.016904,-0.017000,0.017676,-0.000471,0.000471
Day_Charge,0.040085,0.004483,-0.013974,0.043312,1.000000,-0.009042,1.000000,-0.031551,0.002803,-0.062544,...,-0.031423,-0.026267,-0.012290,0.053851,0.144827,-0.025205,0.004093,0.017436,-0.043194,0.043194
Eve_Minutes,-0.004727,0.022186,-0.004055,0.014364,-0.031549,0.012167,-0.031551,1.000000,0.010385,0.541874,...,0.009504,0.011488,-0.001887,-0.014680,0.031011,0.037709,0.000896,-0.006583,-0.017805,0.017805
Eve_Calls,0.007632,-0.006600,0.002685,-0.000770,0.002802,-0.006315,0.002803,0.010385,1.000000,-0.001870,...,-0.000362,-0.004080,-0.004476,0.008184,-0.017782,-0.010624,0.009667,0.018616,0.006699,-0.006699
Eve_Charge,0.013831,0.022284,-0.004202,0.026448,-0.062548,-0.005810,-0.062544,0.541874,-0.001870,1.000000,...,0.001660,0.019697,0.006476,0.003807,0.049928,0.019255,-0.012917,0.010838,-0.043022,0.043022


In [180]:
X = df_train_encoded.drop(['Attrition', 'User ID', "User Job",	"Duration"], axis=1)
y = df_train_encoded['Attrition']

In [181]:
# Let's scale the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_train_scaled, columns = X.columns)

In [182]:
X.head()

,Voicemail_Messages,Day_Minutes,Day_Calls,Day_Charge,Eve_Minutes,Eve_Calls,Eve_Charge,Night_Minutes,Night_Calls,Night_Charge,Intl_Minutes,Intl_Calls,Intl_Charge,Service_Calls,age,Intl_Plan_no,Intl_Plan_yes,Voicemail_Plan_no,Voicemail_Plan_yes
0,0.0,0.880228,0.527607,0.880187,0.749862,0.576471,0.749919,0.498987,0.662857,0.499156,0.568528,0.263158,0.567669,0.375,0.571429,1.0,0.0,1.0,0.0
1,0.0,0.454054,0.625767,0.453983,0.587617,0.700000,0.587642,0.383291,0.474286,0.383230,0.639594,0.368421,0.000000,0.625,0.380952,1.0,0.0,1.0,0.0
2,0.0,0.502987,0.000000,0.503012,0.541736,0.523529,0.541789,0.588861,0.862857,0.589195,0.664975,0.263158,0.000000,0.125,0.047619,0.0,1.0,1.0,0.0
3,0.0,0.328876,0.570552,0.328815,0.000000,0.458824,0.451057,0.636203,0.594286,0.636466,0.492386,0.052632,0.492481,0.250,0.460317,0.0,1.0,1.0,0.0
4,0.0,0.743101,0.496933,0.742972,0.577944,0.764706,0.577886,0.711646,0.691429,0.711874,0.573604,0.052632,0.573308,0.250,0.761905,0.0,0.0,1.0,0.0


In [183]:
# Let's create train and validation datasets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2023, stratify=y)

In [184]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((3320, 19), (830, 19), (3320,), (830,))

In [185]:
y_train.value_counts()

0    3154
1     166
Name: Attrition, dtype: int64

In [186]:
# Let's create a function to train and evaluate the model
def train_model(model, X_t, y_t, X_v, y_v):
    model.fit(X_t, y_t)
    pred = model.predict(X_v)
    print('Accuracy Score: ', accuracy_score(y_v, pred))

In [187]:
# Let's train Logistic Regression model
lr = LogisticRegression()
train_model(lr, X_train, y_train, X_val, y_val)

Accuracy Score:  0.9518072289156626


In [188]:
# Let's train Decision Tree model
dt = DecisionTreeClassifier()
train_model(dt, X_train, y_train, X_val, y_val)

Accuracy Score:  0.9590361445783132


In [189]:
# Let's train Random Forest model
rf = RandomForestClassifier()
train_model(rf, X_train, y_train, X_val, y_val)

Accuracy Score:  0.9698795180722891


In [190]:
# Let's scale the test dataset before making predictions
scaler = MinMaxScaler()
X_test_scaled = scaler.fit_transform(df_test_encoded.drop(columns=['User ID', "User Job",	"Duration"]))
X_test = pd.DataFrame(X_test_scaled, columns = df_test_encoded.drop(columns=['User ID', "User Job",	"Duration"]).columns)

In [191]:
X_test.head()

,Voicemail_Messages,Day_Minutes,Day_Calls,Day_Charge,Eve_Minutes,Eve_Calls,Eve_Charge,Night_Minutes,Night_Calls,Night_Charge,Intl_Minutes,Intl_Calls,Intl_Charge,Service_Calls,age,Intl_Plan_no,Intl_Plan_yes,Voicemail_Plan_no,Voicemail_Plan_yes
0,0.000000,0.368016,0.830303,0.368042,0.593383,0.303279,0.593418,0.482944,0.594828,0.483378,0.635,0.30,0.635185,0.444444,0.031746,1.0,0.0,1.0,0.0
1,0.000000,0.948974,0.406061,0.948860,0.861955,0.418033,0.861996,0.338719,0.741379,0.339096,0.270,0.45,0.270370,0.444444,0.746032,1.0,0.0,1.0,0.0
2,0.000000,0.177879,0.539394,0.177901,0.417143,0.614754,0.417197,0.485338,0.189655,0.485372,0.285,0.30,0.285185,0.555556,0.650794,1.0,0.0,1.0,0.0
3,0.000000,0.711517,0.715152,0.711435,0.665263,0.598361,0.665251,0.696589,0.413793,0.696809,0.590,0.15,0.590741,0.111111,0.888889,1.0,0.0,1.0,0.0
4,0.854167,0.493444,0.515152,0.493461,0.519398,0.500000,0.519462,0.223818,0.310345,0.224069,0.730,0.75,0.729630,0.000000,0.555556,0.0,1.0,0.0,1.0


In [192]:
rf = RandomForestClassifier()
rf.fit(X, y)

RandomForestClassifier()

In [193]:
# Let's make predictions on the test dataset
pred = rf.predict(X_test)

In [194]:
rf.feature_importances_

array([0.01556475, 0.14917538, 0.02967907, 0.13663242, 0.06899428,
       0.03330345, 0.07076906, 0.04449515, 0.03843807, 0.05840831,
       0.05056278, 0.06172061, 0.03933415, 0.07628064, 0.03768299,
       0.0090511 , 0.03999134, 0.01778025, 0.02213619])

In [195]:
sample_submission['Attrition'] = pred

In [196]:
# Let's convert the Attrition column to "Yes" and "No"
sample_submission['Attrition'] = sample_submission['Attrition'].apply(lambda x: 'Yes' if x==1 else 'No')
sample_submission.head()

,User ID,Attrition
0,11,Yes
1,16,Yes
2,22,Yes
3,34,No
4,42,Yes


In [197]:
sample_submission.Attrition.value_counts()

No     602
Yes    248
Name: Attrition, dtype: int64

In [170]:
sample_submission.to_csv('submission.csv', index=False)